In [1]:
import pandas as pd 
import numpy as np 
from demand import prepare_data, prepare_demand_function, fit_demand_function, prepare_predictors
from discounts import apply_discount
transactions = pd.read_csv("data/transactions.csv")
discounts_per_day = pd.read_csv("nr_discounts_per_day.csv")#csv with the nr of discounts daily

In [2]:
product = "Biologische courgette"

filtered_transactions = transactions[(transactions["description"] == product)] #& (transactions["size"] == "150g")]

In [3]:
compl_prod = "Biologisch volkoren heel"

prepared_transactions = prepare_data(filtered_transactions, discounts_per_day, compl_prod, transactions)
prepared_transactions.head()

c:\Users\zolyo\OneDrive\Documents\Quartile 3\JBG050 Data Challenge 2\Code\src\demand.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_transactions["day"] = pd.to_datetime(filtered_transactions["day"], dayfirst=True)
c:\Users\zolyo\OneDrive\Documents\Quartile 3\JBG050 Data Challenge 2\Code\src\demand.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_transactions["discount"] = 100-(
compl_product is a string
c:\Users\zolyo\OneDrive\Documents\Quartile 3\JBG050 Data Challenge 2\Code\src

count   discount  std        min        25%  \
description           day                                                       
Biologische courgette 2018-01-01    8.0  14.792899  0.0  14.792899  14.792899   
                      2018-01-02    1.0  14.792899  NaN  14.792899  14.792899   
                      2018-01-03    2.0  14.792899  0.0  14.792899  14.792899   
                      2018-01-04    2.0  14.792899  0.0  14.792899  14.792899   
                      2018-01-05    3.0  14.792899  0.0  14.792899  14.792899   

                                        50%        75%        max    month  \
description           day                                                    
Biologische courgette 2018-01-01  14.792899  14.792899  14.792899  January   
                      2018-01-02  14.792899  14.792899  14.792899  January   
                      2018-01-03  14.792899  14.792899  14.792899  January   
                      2018-01-04  14.792899  14.792899  14.792899  January   
                      2018-01-05  14.792899  14.792899  14.792899  January   

                                                product  day  dayofweek  \
description           day                                                 
Biologische courgette 2018-01-01  Biologische courgette    1     Monday   
                      2018-01-02  Biologische courgette    2    Tuesday   
                      2018-01-03  Biologische courgette    3  Wednesday   
                      2018-01-04  Biologische courgette    4   Thursday   
                      2018-01-05  Biologische courgette    5     Friday   

                                  purchase_price  std_sales_price  \
description           day                                           
Biologische courgette 2018-01-01            1.44             1.69   
                      2018-01-02            1.44             1.69   
                      2018-01-03            1.44             1.69   
                      2018-01-04            1.44             1.69   
                      2018-01-05            1.44             1.69   

                                  prev_day_purchases  sales      days  \
description           day                                               
Biologische courgette 2018-01-01                 0.0   2.34  1/1/2018   
                      2018-01-02                 8.0   2.34  2/1/2018   
                      2018-01-03                 1.0   2.34  3/1/2018   
                      2018-01-04                 2.0   2.34  4/1/2018   
                      2018-01-05                 2.0   2.34  5/1/2018   

                                  on_discount  
description           day                      
Biologische courgette 2018-01-01           15  
                      2018-01-02           13  
                      2018-01-03            8  
                      2018-01-04           16  
                      2018-01-05           16

In [4]:
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, StackingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.tree import DecisionTreeRegressor

output_dct = prepare_demand_function(prepared_transactions)
nn =  MLPRegressor(hidden_layer_sizes=(50,50))
rf = RandomForestRegressor(n_estimators=50, max_depth=9)
rf2 = RandomForestRegressor(n_estimators=300)
ridge = Ridge()
voter = VotingRegressor([("nn",nn), ("df",rf), ("lin", ridge)])
model = fit_demand_function(output_dct,ridge)

Model fitted; test set metrics: MAE: 1.5921128891019827 MSE: 6.048808476020343, R^2 score: 0.07163580688637294


In [5]:
pd.DataFrame({"predictor":output_dct["train"][0].columns, "coef":model.coef_})

,predictor,coef
0,discount,0.109863
1,purchase_price,-0.001857
2,prev_day_purchases,0.035822
3,on_discount,-0.064755
4,sales,-0.203398
5,x0_Friday,0.408356
6,x0_Monday,0.156941
7,x0_Saturday,1.097835
8,x0_Sunday,-0.295566
9,x0_Thursday,-1.106784


In [6]:
prepared_transactions.dropna(inplace=True)
one_row = prepared_transactions#[prepared_transactions["product"] == "Biologische knoflook"].tail(500)
pred_columns = ["discount", "purchase_price", "prev_day_purchases", "on_discount", "sales"]
target_col = "count"


df_dayofweek = pd.DataFrame(output_dct["weekday_enc"].transform(one_row[["dayofweek"]]), columns=output_dct["weekday_enc"].get_feature_names())

df_month = pd.DataFrame(output_dct["months_enc"].transform(one_row[["month"]]), columns=output_dct["months_enc"].get_feature_names())

#df_product = pd.DataFrame(output_dct["product_enc"].transform(one_row[["product"]]), columns=output_dct["product_enc"].get_feature_names())

X_pred = pd.concat([pd.DataFrame(one_row[pred_columns].values, columns=one_row[pred_columns].columns), df_dayofweek, df_month], axis=1) #add df_product back

one_row["predicted"] = model.predict(X_pred)

In [7]:
import plotly.express as px
fig = px.line(one_row, x="day", y="count")
fig.add_scatter(x=one_row["day"], y=one_row["predicted"], mode="lines")
fig.show()

In [8]:
modded = one_row.tail(1)
#modded["discount"] = 30
X_mod = prepare_predictors(modded, output_dct)
model.predict(X_mod)

c:\Users\zolyo\OneDrive\Documents\Quartile 3\JBG050 Data Challenge 2\Code\src\demand.py:143: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



array([3.84962912])

In [13]:
X_mod

,discount,purchase_price,prev_day_purchases,on_discount,sales,x0_Friday,x0_Monday,x0_Saturday,x0_Sunday,x0_Thursday,...,x0_December,x0_February,x0_January,x0_July,x0_June,x0_March,x0_May,x0_November,x0_October,x0_September
0,0.0,1.69,3.0,13.0,2.34,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
model.decision_path()

AttributeError: 'Ridge' object has no attribute 'decision_path'

In [12]:
X_mod20 = apply_discount(X_mod, 35)
model.predict(X_mod20)

array([7.63116127])

In [21]:
modded

,,count,discount,std,min,25%,50%,75%,max,month,product,day,dayofweek,purchase_price,std_sales_price,prev_day_purchases,sales,days,on_discount,predicted
description,day,,,,,,,,,,,,,,,,,,,
Biologische courgette,2018-12-30,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,December,Biologische courgette,364,Sunday,1.69,1.69,3.0,2.34,30-12-2018,13,3.65273


In [47]:
import plotly.express as px
fig = px.scatter(one_row, x="purchase_price", y="count")
fig.show()

In [39]:
one_row

count   discount  std        min        25%  \
description          day                                                       
Biologische knoflook 2018-01-01    6.0  30.434783  0.0  30.434783  30.434783   
                     2018-01-03    6.0   0.000000  0.0   0.000000   0.000000   
                     2018-01-05    6.0   4.347826  0.0   4.347826   4.347826   
                     2018-01-06    5.0   0.000000  0.0   0.000000   0.000000   
                     2018-01-07    7.0  30.434783  0.0  30.434783  30.434783   
...                                ...        ...  ...        ...        ...   
                     2018-12-25    5.0   0.000000  0.0   0.000000   0.000000   
                     2018-12-26    7.0   0.000000  0.0   0.000000   0.000000   
                     2018-12-28    4.0   0.000000  0.0   0.000000   0.000000   
                     2018-12-29    2.0   0.000000  0.0   0.000000   0.000000   
                     2018-12-30    2.0   0.000000  0.0   0.000000   0.000000   

                                       50%        75%        max     month  \
description          day                                                     
Biologische knoflook 2018-01-01  30.434783  30.434783  30.434783   January   
                     2018-01-03   0.000000   0.000000   0.000000   January   
                     2018-01-05   4.347826   4.347826   4.347826   January   
                     2018-01-06   0.000000   0.000000   0.000000   January   
                     2018-01-07  30.434783  30.434783  30.434783   January   
...                                    ...        ...        ...       ...   
                     2018-12-25   0.000000   0.000000   0.000000  December   
                     2018-12-26   0.000000   0.000000   0.000000  December   
                     2018-12-28   0.000000   0.000000   0.000000  December   
                     2018-12-29   0.000000   0.000000   0.000000  December   
                     2018-12-30   0.000000   0.000000   0.000000  December   

                                              product  day  dayofweek  \
description          day                                                
Biologische knoflook 2018-01-01  Biologische knoflook    1     Monday   
                     2018-01-03  Biologische knoflook    3  Wednesday   
                     2018-01-05  Biologische knoflook    5     Friday   
                     2018-01-06  Biologische knoflook    6   Saturday   
                     2018-01-07  Biologische knoflook    7     Sunday   
...                                               ...  ...        ...   
                     2018-12-25  Biologische knoflook  359    Tuesday   
                     2018-12-26  Biologische knoflook  360  Wednesday   
                     2018-12-28  Biologische knoflook  362     Friday   
                     2018-12-29  Biologische knoflook  363   Saturday   
                     2018-12-30  Biologische knoflook  364     Sunday   

                                 purchase_price  std_sales_price  discount_2  \
description          day                                                       
Biologische knoflook 2018-01-01            0.48             0.69  926.275992   
                     2018-01-03            0.69             0.69    0.000000   
                     2018-01-05            0.66             0.69   18.903592   
                     2018-01-06            0.69             0.69    0.000000   
                     2018-01-07            0.48             0.69  926.275992   
...                                         ...              ...         ...   
                     2018-12-25            0.69             0.69    0.000000   
                     2018-12-26            0.69             0.69    0.000000   
                     2018-12-28            0.69             0.69    0.000000   
                     2018-12-29            0.69             0.69    0.000000   
                     2018-12-30            0.69             0.69   

In [25]:
transactions["on_discount"] = np.where(transactions["std_sales_price"] == transactions["purchase_price"], 0, 1)
grouped_transactions = transactions.groupby(["description", "day"]).describe()
grouped_transactions

customer                                              \
                          count          mean        std      min       25%   
description  day                                                              
Aardbeien    1/1/2018       9.0     38.333333  16.000000      7.0     31.00   
             1/10/2018      6.0  15646.833333  23.819460  15621.0  15625.75   
             1/11/2018      6.0  17345.666667  11.075498  17333.0  17337.50   
             1/12/2018      5.0  19071.400000  11.436783  19054.0  19071.00   
             1/2/2018       8.0   1720.375000  13.037720   1704.0   1709.50   
...                         ...           ...        ...      ...       ...   
rundervinken 9/5/2018       1.0   7448.000000        NaN   7448.0   7448.00   
             9/6/2018       7.0   9285.428571  17.709293   9257.0   9276.50   
             9/7/2018       7.0  10946.142857  19.038807  10914.0  10936.50   
             9/8/2018       6.0  12685.833333  10.457852  12669.0  12681.00   
             9/9/2018       2.0  14419.500000   6.363961  14415.0  14417.25   

                                                   bank acount                \
                            50%       75%      max       count          mean   
description  day                                                               
Aardbeien    1/1/2018      39.0     46.00     60.0         9.0  6.196480e+07   
             1/10/2018  15649.0  15667.00  15671.0         6.0  7.127359e+07   
             1/11/2018  17348.0  17348.00  17363.0         6.0  4.081008e+07   
             1/12/2018  19072.0  19074.00  19086.0         3.0  3.525882e+07   
             1/2/2018    1721.0   1729.75   1740.0         7.0  6.307131e+07   
...                         ...       ...      ...         ...           ...   
rundervinken 9/5/2018    7448.0   7448.00   7448.0         1.0  7.380519e+07   
             9/6/2018    9288.0   9296.50   9307.0         5.0  3.724918e+07   
             9/7/2018   10947.0  10959.50  10970.0         7.0  5.381642e+07   
             9/8/2018   12687.5  12692.50  12698.0         4.0  3.107016e+07   
             9/9/2018   14419.5  14421.75  14424.0         2.0  2.884079e+07   

                        ... basic      on_discount                           \
                        ...   75%  max       count mean  std  min  25%  50%   
description  day        ...                                                   
Aardbeien    1/1/2018   ...   0.0  0.0         9.0  0.0  0.0  0.0  0.0  0.0   
             1/10/2018  ...   0.0  0.0         6.0  0.0  0.0  0.0  0.0  0.0   
             1/11/2018  ...   0.0  0.0         6.0  0.0  0.0  0.0  0.0  0.0   
             1/12/2018  ...   0.0  0.0         5.0  0.0  0.0  0.0  0.0  0.0   
             1/2/2018   ...   0.0  0.0         8.0  1.0  0.0  1.0  1.0  1.0   
...                     ...   ...  ...         ...  ...  ...  ...  ...  ...   
rundervinken 9/5/2018   ...   0.0  0.0         1.0  0.0  NaN  0.0  0.0  0.0   
             9/6/2018   ...   0.0  0.0         7.0  0.0  0.0  0.0  0.0  0.0   
             9/7/2018   ...   0.0  0.0         7.0  0.0  0.0  0.0  0.0  0.0   
             9/8/2018   ...   0.0  0.0         6.0  0.0  0.0  0.0  0.0  0.0   
             9/9/2018   ...   0.0  0.0         2.0  0.0  0.0  0.0  0.0  0.0   

                                  
                        75%  max  
description  day                  
Aardbeien    1/1/2018   0.0  0.0  
             1/10/2018  0.0  0.0  
             1/11/2018  0.0  0.0  
             1/12/2018  0.0  0.0  
             1/2/2018   1.0  1.0  
...                     ...  ...  
rundervinken 9/5/2018   0.0  0.0  
             9/6/2018   0.0  0.0  
             9/7/2018   0.0  0.0  
             9/8/2018   0.0  0.0  
             9/9/2018   0.0  0.0  

[53181 rows x 56 columns]

In [45]:
grouped_transactions["day"] = [grouped_transactions.index[i][1] for i in range(grouped_transactions.shape[0])]
grouped_transactions["on_discount"] = np.where(grouped_transactions["std_sales_price"]["mean"] == grouped_transactions["purchase_price"]["mean"], 0, 1)
grouped_transactions["on_discount"]["count"]

df_discounts = pd.DataFrame({"days":[grouped_transactions.index[i][1] for i in range(grouped_transactions.shape[0])], "on_discount":grouped_transactions["on_discount"]["count"]})
discounts_per_day = df_discounts.groupby("days").sum()
discounts_per_day["day"] = pd.to_datetime(discounts_per_day.index, dayfirst=True)
discounts_per_day.to_csv("nr_discounts_per_day.csv")

In [49]:
new_df = prepared_transactions.join(discounts_per_day.set_index("day"))

In [50]:
new_df

count  discount  std  min  25%  50%  75%  \
description           day                                                    
Biologische Bloemkool 2018-01-01    5.0       0.0  0.0  0.0  0.0  0.0  0.0   
                      2018-01-04    4.0       0.0  0.0  0.0  0.0  0.0  0.0   
                      2018-01-05    6.0       0.0  0.0  0.0  0.0  0.0  0.0   
                      2018-01-07    9.0       0.0  0.0  0.0  0.0  0.0  0.0   
                      2018-01-08    2.0       0.0  0.0  0.0  0.0  0.0  0.0   
...                                 ...       ...  ...  ...  ...  ...  ...   
Biologische rode kool 2018-12-25    3.0       0.0  0.0  0.0  0.0  0.0  0.0   
                      2018-12-26    7.0       0.0  0.0  0.0  0.0  0.0  0.0   
                      2018-12-28    5.0       0.0  0.0  0.0  0.0  0.0  0.0   
                      2018-12-29    3.0       0.0  0.0  0.0  0.0  0.0  0.0   
                      2018-12-30    3.0       0.0  0.0  0.0  0.0  0.0  0.0   

                                  max     month                product  day  \
description           day                                                     
Biologische Bloemkool 2018-01-01  0.0   January  Biologische Bloemkool    1   
                      2018-01-04  0.0   January  Biologische Bloemkool    4   
                      2018-01-05  0.0   January  Biologische Bloemkool    5   
                      2018-01-07  0.0   January  Biologische Bloemkool    7   
                      2018-01-08  0.0   January  Biologische Bloemkool    8   
...                               ...       ...                    ...  ...   
Biologische rode kool 2018-12-25  0.0  December  Biologische rode kool  359   
                      2018-12-26  0.0  December  Biologische rode kool  360   
                      2018-12-28  0.0  December  Biologische rode kool  362   
                      2018-12-29  0.0  December  Biologische rode kool  363   
                      2018-12-30  0.0  December  Biologische rode kool  364   

                                  dayofweek  purchase_price  discount_2  \
description           day                                                 
Biologische Bloemkool 2018-01-01     Monday            2.39         0.0   
                      2018-01-04   Thursday            2.39         0.0   
                      2018-01-05     Friday            2.39         0.0   
                      2018-01-07     Sunday            2.39         0.0   
                      2018-01-08     Monday            2.39         0.0   
...                                     ...             ...         ...   
Biologische rode kool 2018-12-25    Tuesday            1.29         0.0   
                      2018-12-26  Wednesday            1.29         0.0   
                      2018-12-28     Friday            1.29         0.0   
                      2018-12-29   Saturday            1.29         0.0   
                      2018-12-30     Sunday            1.29         0.0   

                                  prev_day_purchases  prev_day_pp  \
description           day                                           
Biologische Bloemkool 2018-01-01                 0.0          NaN   
                      2018-01-04                 1.0         2.39   
                      2018-01-05                 4.0         2.39   
                      2018-01-07                 1.0         2.39   
                      2018-01-08                 9.0         2.39   
...                                              ...          ...   
Biologische rode kool 2018-12-25                 4.0         1.29   
                      2018-12-26                 3.0         1.29   
                      2018-12-28                 1.0         1.29   
                      2018-12-29                 5.0         1.29   
                      2018-12-30                 3.0         1.29   

                                  discount_prev_day  on_discount  
description           day                         

## Experimenting with TensorFlow

In [59]:
from tensorflow.keras import models, layers, losses, optimizers

lstm_model = models.Sequential()
lstm_model.add(layers.LSTM(50, return_sequences=True))
lstm_model.add(layers.LSTM(20))
lstm_model.add(layers.Dense(1, activation="sigmoid"))
lstm_model.compile(loss="adam", metrics="mean_absolute_error")

In [62]:
X_train, y_train = output_dct["train"][0], output_dct["train"][1]
lstm_model.fit(X_train, y_train, batch_size=16, epochs=30)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).